In [46]:
######################################################### Imports #############################################################

import vrep
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

In [47]:
################################################## Conexão com o V-rep #######################################################

clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # Faz conexão, se retorna -1 é que deu erro

if clientID!=-1:
    print('Conexão sucedida')
else:
    print("Erro na conexão")

Erro na conexão


In [48]:
################################################ Declaração de Váriáveis ######################################################
   
    # Função np.arrange dá sequenência de numeros, dado umintervalo
    # ctrl.Antecedent descrita na documentação do scikit-fuzzy,idem à  ctrl.Consequent
    
## Valores Crisp
sensores_handle = []        #lista com os Handles dos sensores, para pegar os Handles mais fácil
valores_sensores = []       # Lista com os valores dos sensores


# Valores Fuzzy
    # Antecedentes: Váriaveis à serem Fuzificada
sensor_frente  = ctrl.Antecedent(np.arange(0,0.12, 0.01),'sensor_frente') #Vai até 0,1 Metro, pelo que eu vi na simulação
sensores_left  = ctrl.Antecedent(np.arange(0,0.12, 0.01), 'sensores_left')
sensores_right = ctrl.Antecedent(np.arange(0,0.12, 0.01), 'sensores_right')

    # Consequentes
v_left  = ctrl.Consequent(np.arange(0, 10, 0.5), 'v_left')         # velocidade motor esquerdo, vai de 0 à 10 (eu determinei impiricamente na simulação)
v_right = ctrl.Consequent(np.arange(0, 10, 0.5), 'v_right')        # idem ao de cima pro motor direito

In [49]:
############################################ Declaração de Handles ############################################################

# Não sei direito o que é um handle até hoje, mas chutaria que ele é uma coisa que controla o objeto real

#handle do motor esquerdo, chuto que o nome do objeto seja o mesmo da simulação
erro_code, leftMotorHandle = vrep.simxGetObjectHandle(clientID,'KJunior_wheelLeft', vrep.simx_opmode_oneshot_wait )

#Handle do motor Direito
erro_code, rightMotorHandle = vrep.simxGetObjectHandle(clientID,'KJunior_wheelLeft', vrep.simx_opmode_oneshot_wait )

#Handle dos sensores, só importam os do 1 ao 5, o 3 é o sensor mais centralizado
for i in range(5):
    erro_code, sensor_handle = vrep.simxGetObjectHandle(clientID,'KJunior_proxSensor%d' % (i+1) , vrep.simx_opmode_oneshot_wait)
    sensores_handle.append(sensor_handle)

    # Não sei porque, mas a primeira leitura tem que ser assim
    error_code,estado,coord,detectedObjectHandle,detectedSurfaceNormalVector = vrep.simxReadProximitySensor(clientID, sensor_handle,vrep.simx_opmode_streaming)

In [50]:
############################################# Lógica Fuzzy ################################################################

## Funções de pertinencia (Também conhecidas como membership)
    # Fuzz.trimf faz uma função de pertinencia triangulas onde [x,y,z] são os pontos do triangulo
    
# Sensor da frente
sensor_frente['perto'] = fuzz.trimf(sensor_frente.universe, [0, 0, 0.065])
sensor_frente['longe'] = fuzz.trimf(sensor_frente.universe,[0.065,0.065,0.12])

# Sensor do lado esquerdo
sensores_left['perto'] = fuzz.trimf(sensores_left.universe, [0, 0, 0.065])
sensores_left['longe'] = fuzz.trimf(sensores_left.universe,[0.065,0.065,0.12])

# Sensores do lado direito
sensores_right['perto'] = fuzz.trimf(sensores_right.universe, [0, 0, 0.065])
sensores_right['longe'] = fuzz.trimf(sensores_right.universe,[0.065,0.065,0.12])

# Velocidade Motor esquerdo 
v_left['parado'] = fuzz.trimf(v_left.universe, [0,0,0.001])
v_left['lento'] = fuzz.trimf(v_left.universe, [0.001,0.05,0.1])
v_left['rapido'] = fuzz.trimf(v_left.universe, [0.05,0.1,0.1])

# Velocidade motor direito
v_right['parado'] = fuzz.trimf(v_right.universe, [0,0,0.001])
v_right['lento'] = fuzz.trimf(v_right.universe, [0.001,0.05,0.1])
v_right['rapido'] = fuzz.trimf(v_right.universe, [0.05,0.1,0.1])

## Regras Fuzzy